<a href="https://colab.research.google.com/github/yoyochrist/Thesis/blob/master/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import time
import numpy as np # linear algebra
from numpy.testing import assert_allclose
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive
from google.colab import files
import json
import io
import os

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
drive.mount("/content/gdrive/")
print(os.listdir("/content/gdrive/My Drive/Thesis/input"))
root_path='/content/gdrive/My Drive/Thesis/input/'

# Any results you write to the current directory are saved as output.

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/
['master-raw.xlsx', 'master-raw.csv', 'wiki.id.vec', 'summaries', 'checkpoints', 'logs', 'train_x_res.csv', 'test_x_res.csv', 'valid_x.csv', 'train_y_res.csv', 'test_y_res.csv', 'valid_y.csv']


In [0]:
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Input, LSTM, GRU, Conv1D, MaxPooling1D, Concatenate, Bidirectional, Flatten, Activation, RepeatVector, Permute, multiply
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from keras.utils import np_utils
from keras.initializers import glorot_uniform
from keras.callbacks import Callback
import keras.backend as K
from keras.optimizers import Adam

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
maxLen = 128          #maximum length of sentences

import string
tr = str.maketrans(string.punctuation, " "*32)
def modify_phrase(ph, tr):
    ph = ph.lower()
    return ph.translate(tr).strip()

In [0]:
modify_phrase("Hey there! I am here, using whatsapp!!...", tr)

'hey there  i am here  using whatsapp'

In [0]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    vocab_size, dim = map(int, fin.readline().split())
    word_to_vec_map = {}
    words = set()
    for line in fin:
        tokens = line.rstrip().split(' ')
        words.add(tokens[0])
        word_to_vec_map[tokens[0]] = np.array(tokens[1:], dtype=np.float64)
    i = 1
    words_to_index = {}
    index_to_words = {}
    for w in sorted(words):
        words_to_index[w] = i
        index_to_words[i] = w
        i = i + 1
    return word_to_vec_map, words_to_index, index_to_words, vocab_size, dim

In [0]:
def sentences_to_indices(X, word_to_index, maxLen):
    m = X.shape[0]                                   # number of training examples
    X_indices = np.zeros((m, maxLen))
    
    for i in range(m):
        sentence_words = X[i].lower().strip().split()
        j = 0
        for w in sentence_words:
            if w not in word_to_index:
                w = "person"        #mostly names are not present in vocabulary
            X_indices[i, j] = word_to_index[w]
            if j < maxLen-1:
              j = j + 1
            else:
              break
    
    return X_indices

In [0]:
#load fastText word2vec model
word_to_vec_map, word_to_index, index_to_words, vocab_size, dim= load_vectors('/content/gdrive/My Drive/Thesis/input/wiki.id.vec')

In [0]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(input_dim = vocab_len, output_dim = emb_dim, trainable = False) 

    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [0]:
def restaurant_review_analysis(input_shape, word_to_vec_map, word_to_index, method, attention):
    sentence_indices = Input(shape=input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    embeddings = embedding_layer(sentence_indices)
    
    if attention == True:
      activations = LSTM(units=128, dropout=0.4, return_sequences=True)(embeddings)     
      attention = Dense(1, activation='tanh')(activations)
      attention = Flatten()(attention)
      attention = Activation('relu')(attention)
      attention = RepeatVector(128)(attention)
      attention = Permute([2, 1])(attention)

      sent_representation = multiply([activations, attention])
    else:
      sent_representation = embeddings
      
    if method == 'CNN':
      model = Sequential()
      X1 = Conv1D(128, 64)(sent_representation)
      X2 = Conv1D(128, 64)(sent_representation)
      X3 = Conv1D(128, 64)(sent_representation)
      X1 = MaxPooling1D(pool_size=32)(X1)
      X2 = MaxPooling1D(pool_size=32)(X2)
      X3 = MaxPooling1D(pool_size=32)(X3)
      X = Concatenate(axis=1)([X1, X2, X3])
      X = Flatten()(X)
      X = Dense(units=4, activation='relu')(X)
    elif method == 'LSTM':
      #X = LSTM(units=128, dropout=0.4)(sent_representation)
      X = LSTM(units=128, dropout=0.3, return_sequences=True)(sent_representation)
      X = Flatten()(X)
      X = Dense(units=4, activation='relu')(X)
    else:
      X = Bidirectional(LSTM(units=128, dropout=0.3, return_sequences=True))(sent_representation)
      X = Flatten()(X)
      X = Dense(units=4, activation='relu')(X)
    
    model = Model(inputs=sentence_indices, outputs=X)
    return model

In [0]:
# model_cnn = restaurant_review_analysis((maxLen,), word_to_vec_map, word_to_index, "CNN", False)
# model_lstm = restaurant_review_analysis((maxLen,), word_to_vec_map, word_to_index, "LSTM", False)
# model_blstm = restaurant_review_analysis((maxLen,), word_to_vec_map, word_to_index, "BiLSTM", False)
# model_cnn_att = restaurant_review_analysis((maxLen,), word_to_vec_map, word_to_index, "CNN", True)
# model_lstm_att = restaurant_review_analysis((maxLen,), word_to_vec_map, word_to_index, "LSTM", True)
model_blstm_att = restaurant_review_analysis((maxLen,), word_to_vec_map, word_to_index, "BiLSTM", True)

W0731 03:03:40.527044 140037968222080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0731 03:03:40.566877 140037968222080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0731 03:03:41.663922 140037968222080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0731 03:03:41.680233 140037968222080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0731 03:03:41.681321 1400379682

In [0]:
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [0]:
def f1_score(y_true, y_pred):
  # Count positive samples.
  c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
  c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

  # If there are no true samples, fix the F1 score at 0.
#   if c3 == 0:
#     return 0

  # How many selected items are relevant?
  precision = c1 / c2

  # How many relevant items are selected?
  recall = c1 / c3

  # Calculate f1_score
  f1_score = 2 * (precision * recall) / (precision + recall)
  return f1_score

In [0]:
# model_cnn.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy",f1_score])
# model_cnn.compile(optimizer = Adam(lr=0.0005), loss = "categorical_crossentropy", metrics = ["accuracy",f1_score])
# model_lstm.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy",f1_score])
# model_blstm.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy",f1_score])
# model_cnn_att.compile(optimizer = Adam(lr=0.0005), loss = "categorical_crossentropy", metrics = ["accuracy",f1_score])
# model_lstm_att.compile(optimizer = Adam(lr=0.0005), loss = "categorical_crossentropy", metrics = ["accuracy",f1_score])
model_blstm_att.compile(optimizer = Adam(lr=0.0005), loss = "categorical_crossentropy", metrics = ["accuracy",f1_score])

earlystop = EarlyStopping(monitor='acc', min_delta=0.000001, patience=10, \
                          mode='auto')
time_callback = TimeHistory()

W0731 03:03:50.892448 140037968222080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
from contextlib import redirect_stdout

# with open(root_path+'summaries/model_cnn_summary.txt', 'w+') as f:
#   with redirect_stdout(f):
#     model_cnn.summary()

# with open(root_path+'summaries/model_lstm_summary.txt', 'w+') as f:
#   with redirect_stdout(f):
#     model_lstm.summary()
    
# with open(root_path+'summaries/model_blstm_summary.txt', 'w+') as f:
#   with redirect_stdout(f):
#     model_blstm.summary()

# with open(root_path+'summaries/model_cnn_att_summary.txt', 'w+') as f:
#   with redirect_stdout(f):
#     model_cnn_att.summary()

# with open(root_path+'summaries/model_lstm_att_summary.txt', 'w+') as f:
#   with redirect_stdout(f):
#     model_lstm_att.summary()
    
with open(root_path+'summaries/model_blstm_att_summary.txt', 'w+') as f:
  with redirect_stdout(f):
    model_blstm_att.summary()

In [0]:
x_train = pd.read_csv(root_path+"train_x_res.csv")
x_test = pd.read_csv(root_path+"test_x_res.csv")
x_valid = pd.read_csv(root_path+"valid_x.csv")

y_train = pd.read_csv(root_path+"train_y_res.csv")
y_test = pd.read_csv(root_path+"test_y_res.csv")
y_valid = pd.read_csv(root_path+"valid_y.csv")

train_x = np.array(x_train)
test_x = np.array(x_test)
valid_x = np.array(x_valid)

train_y = np.array(y_train)
test_y = np.array(y_test)
valid_y = np.array(y_valid)

# x_train = np.genfromtxt(root_path+"train_x_res.csv", delimiter=",")
# x_test = np.genfromtxt(root_path+"test_x_res.csv", delimiter=",")
# x_valid = np.genfromtxt(root_path+"valid_x.csv", delimiter=",")

# y_train = np.genfromtxt(root_path+"train_y_res.csv", delimiter=",")
# y_test = np.genfromtxt(root_path+"test_y_res.csv", delimiter=",")
# y_valid = np.genfromtxt(root_path+"valid_x.csv", delimiter=",")

train_x = train_x[:,1:]
test_x = test_x[:,1:]
valid_x = valid_x[:,1:]

train_y = train_y[:,1:]
test_y = test_y[:,1:]
valid_y = valid_y[:,1:]

In [0]:
train_x.shape

(134937, 128)

In [0]:
checkpoint = ModelCheckpoint(root_path+'checkpoints/model_blstm-3.hdf5', monitor='acc', verbose=1, \
                             save_best_only=True,\
                             mode='min')

In [0]:
csv_logger = CSVLogger(root_path+'logs/training_blstm.log')
# callbacks_list = [earlystop, csv_logger,checkpoint, time_callback]
callbacks_list = [csv_logger,checkpoint, time_callback]
# track_cnn = model_blstm.fit(train_x, train_y, batch_size=128, epochs=70, callbacks=callbacks_list)
track_blstm = model_blstm_att.fit(train_x, train_y, batch_size=128, epochs=30, validation_data=(valid_x, valid_y), callbacks=callbacks_list)

W0731 03:08:02.147935 140037968222080 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 134937 samples, validate on 44980 samples
Epoch 1/30
134937/134937 [==============================] - 682s 5ms/step - loss: 3.8667 - acc: 0.4118 - f1_score: nan - val_loss: 0.7115 - val_acc: 0.6796 - val_f1_score: 0.6006

Epoch 00001: acc improved from inf to 0.41179, saving model to /content/gdrive/My Drive/Thesis/input/checkpoints/model_blstm-3.hdf5
Epoch 2/30
 70912/134937 [==============>...............] - ETA: 4:47 - loss: 0.7372 - acc: 0.6857 - f1_score: 0.6253

In [0]:
# with open(root_path+'logs/history-model-cnn-time.json', 'w') as f:
#     json.dump(time_callback.times, f)

with open(root_path+'logs/history-model-blstm-att-time.json', 'w') as f:
    json.dump(time_callback.times, f)

In [0]:
test_x.shape

In [0]:
csv_logger = CSVLogger(root_path+'logs/training_blstm-att-2.log')
# callbacks_list = [earlystop, csv_logger,checkpoint, time_callback]
callbacks_list = [csv_logger,checkpoint, time_callback]
# load the model
new_model = load_model(root_path+'checkpoints/model_blstm_att-3.hdf5', custom_objects={'f1_score':f1_score})
np.allclose(model_blstm_att.predict(train_x),new_model.predict(train_x),1e-5)
# fit the model
track_blstm_2 = new_model.fit(train_x, train_y, epochs=30, batch_size=128, validation_data=(valid_x, valid_y), callbacks=callbacks_list)

In [0]:
y_train.shape

In [0]:
model_blstm.predict(train_x)

In [0]:
new_model.predict(x_train)

In [0]:
assert_allclose(model_blstm.predict(x_train),new_model.predict(x_train),1e-5)

In [0]:
# with open(root_path+'logs/history-model-cnn-time.json', 'w') as f:
#     json.dump(time_callback.times, f)

with open(root_path+'logs/history-model-blstm-att-time-2.json', 'w') as f:
    json.dump(time_callback.times, f)

In [0]:
csv_logger = CSVLogger(root_path+'logs/training_blstm-att-3.log')
# callbacks_list = [earlystop, csv_logger,checkpoint, time_callback]
callbacks_list = [csv_logger,checkpoint, time_callback]
# load the model
new_model_2 = load_model(root_path+'checkpoints/model_blstm_att-3.hdf5', custom_objects={'f1_score':f1_score})
np.allclose(model_blstm_att.predict(train_x),new_model_2.predict(train_x),1e-5)

# fit the model
track_blstm_3 = new_model_2.fit(train_x, train_y, batch_size=128, epochs=30, validation_data=(valid_x, valid_y), callbacks=callbacks_list)

In [0]:
# with open(root_path+'logs/history-model-cnn-time.json', 'w') as f:
#     json.dump(time_callback.times, f)

with open(root_path+'logs/history-model-blstm-att-time-3.json', 'w') as f:
    json.dump(time_callback.times, f)

In [0]:
csv_logger = CSVLogger(root_path+'logs/training_blstm-att-4.log')
# callbacks_list = [earlystop, csv_logger,checkpoint, time_callback]
callbacks_list = [csv_logger,checkpoint, time_callback]
# load the model
new_model_3 = load_model(root_path+'checkpoints/model_blstm_att-3.hdf5', custom_objects={'f1_score':f1_score})
np.allclose(model_blstm_att.predict(train_x),
                           new_model_3.predict(train_x),
                           1e-5)

# fit the model
track_blstm_4 = new_model_3.fit(train_x, train_y, batch_size=128, epochs=10, validation_data=(valid_x, valid_y), callbacks=callbacks_list)

In [0]:
# with open(root_path+'logs/history-model-cnn-time.json', 'w') as f:
#     json.dump(time_callback.times, f)

with open(root_path+'logs/history-model-blstm-att-time-4.json', 'w') as f:
    json.dump(time_callback.times, f)

In [0]:
# checkpoint = ModelCheckpoint(root_path+'checkpoints/model_blstm-3.hdf5', monitor='acc', verbose=2, \
#                              save_best_only=True, save_weights_only=False,\
#                              mode='auto', period=1)
# csv_logger = CSVLogger(root_path+'logs/training_cnn.log')
# # callbacks_list = [earlystop, csv_logger,checkpoint, time_callback]
# callbacks_list = [csv_logger,checkpoint, time_callback]
# track_cnn = model_blstm.fit(train_x, train_y, batch_size=128, epochs=70, callbacks=callbacks_list)
# # track_cnn = model_cnn.fit(train_x, train_y, batch_size=128, epochs=200, validation_data=(valid_x, valid_y), callbacks=callbacks_list)

In [0]:
# checkpoint = ModelCheckpoint(root_path+'checkpoints/model_lstm-3.hdf5', monitor='acc', verbose=2, \
#                              save_best_only=True, save_weights_only=False,\
#                              mode='auto', period=1)
# csv_logger = CSVLogger(root_path+'logs/training_lstm.log')
# # callbacks_list = [earlystop, csv_logger,checkpoint, time_callback]
# callbacks_list = [csv_logger,checkpoint, time_callback]
# track_lstm = model_lstm.fit(train_x, train_y, batch_size=128, epochs=200, validation_data=(valid_x, valid_y), callbacks=callbacks_list)

In [0]:
plt.plot(track_blstm.history['acc'])
plt.title('BiLSTM Model Accuracy')
plt.ylabel('Accuracy')
plt.ylim(0)
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.savefig(root_path+'graphs/blstm_att_acc_fig-2.png')
plt.show()

In [0]:
# plt.plot(track_cnn.history['loss'])
# plt.title('CNN Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train'], loc='upper left')
# plt.savefig(root_path+'graphs/cnn_loss_fig-2.png')
# plt.show()

plt.plot(track_blstm.history['loss'])
plt.title('BiLSTM Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.savefig(root_path+'graphs/blstm_att_loss_fig-2.png')
plt.show()

In [0]:
# score = model_cnn.evaluate(test_x, test_y, verbose=1)
# print('CNN Test Loss:', score[0])
# print('CNN Test Accuracy:', score[1])

score = model_blstm_att.evaluate(test_x, test_y, verbose=1)
print('BiLSTM Attenntion Test Loss:', score[0])
print('BiLSTM Attenntion Test Accuracy:', score[1])